<img src="https://nlp.johnsnowlabs.com/assets/images/logo.png" width="180" height="50" style="float: left;">

## CRF Named Entity Recognition
In the following example, we walk-through a Conditional Random Fields NER model training and prediction.

This challenging annotator will require the user to provide either a labeled dataset during fit() stage, or use external CoNLL 2003 resources to train. It may optionally use an external word embeddings set and a list of additional entities.

The CRF Annotator will also require Part-of-speech tags so we add those in the same Pipeline. Also, we could use our special RecursivePipeline, which will tell SparkNLP's NER CRF approach to use the same pipeline for tagging external resources.



#### 1. Call necessary imports and set the resource path to read local data files

In [1]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import time
import zipfile

#### 2. Download training dataset if not already there

In [20]:
# Download CoNLL 2003 Dataset
import os
from pathlib import Path
import urllib.request

if not Path("eng.train").is_file():
    print("File Not found will downloading it!")
    url = "https://github.com/patverga/torch-ner-nlp-from-scratch/raw/master/data/conll2003/eng.train"
    urllib.request.urlretrieve(url, 'eng.train')
else:
    print("File already present.")


File Not found will downloading it!


#### 3. Load SparkSession if not already there

In [1]:
import sparknlp 

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.3.4
Apache Spark version:  2.4.3


#### 4. Create annotator components in the right order, with their training Params. Finisher will output only NER. Put all in pipeline.

In [18]:
nerTagger = NerCrfApproach()\
  .setInputCols(["sentence", "token", "pos", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMinEpochs(1)\
  .setMaxEpochs(1)\
  .setLossEps(1e-3)\
  .setL2(1)\
  .setC0(1250000)\
  .setRandomSeed(0)\
  .setVerbose(0)


#### 6. Load a dataset for prediction. Training is not relevant from this dataset.

In [16]:
from sparknlp.training import CoNLL
conll = CoNLL()
data = conll.readDataset(spark, path='eng.train')

embeddings = WordEmbeddingsModel.pretrained()\
.setOutputCol('embeddings')

ready_data = embeddings.transform(data)

ready_data.show(4)

glove_100d download started this may take some time.
Approximate size to download 144.3 MB
[OK!]
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|          embeddings|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|EU rejects German...|[[document, 0, 28...|[[document, 0, 47...|[[token, 0, 1, EU...|[[pos, 0, 1, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|Rare Hendrix song...|[[document, 0, 97...|[[document, 0, 50...|[[token, 0, 3, Ra...|[[pos, 0, 3, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|China says Taiwan...|[[document, 0, 13...|[[document, 0, 46...|[[token, 0, 4, Ch...|[[pos, 0, 4, NNP,...|[[named_entity, 0...|[[word_embeddings...|
|China sa

#### 7. Training the model. Training doesn't really do anything from the dataset itself.

In [19]:
start = time.time()
print("Start fitting")
ner_model = nerTagger.fit(ready_data)
print("Fitting has ended")
print (time.time() - start)

Start fitting
Fitting has ended
154.13315606117249


#### 8. Save NerCrfModel into disk after training

In [ ]:
model.write().overwrite().save("./pip_wo_embedd/")